# Langchain Demo

## Introduction
This is a notebook version of [this tutorial](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/) from langchain's documents. If you want further clarification on some concepts, you can read up on them there.

In this notebook, we will look at the basics of getting a langchain agent working and answering simple questions. It will take you through the process of setting up an LLM API access account for a free LLM endpoint, creating a sample agent, and asking the agent questions all through the use of `langchain`, a python library.

Later notebooks in the series will cover topics such as adding context to the agents, creating teams of agents that can talk to eachother, fetching information from the web for agent context etc.

## Environment Setup
This notebook assumes you already have a python environment on your machine, if this is not the case, refer to `README.md` for instructions on how to get one setup locally. We also assume a basic knowledge of python and programming concepts. Once you have a python environment on your local machine, we need to install a couple of packages in order to use langchain. Run the below command in your terminal to install all the necessary packages:

```
pip install notebook langchain langchain-cohere
```
## Third Party Tools
We will be utilizing Cohere models as they are free (with a rate limit). This allows us to play around in a sandbox without incurring charges like we would if we used something like OpenAI models. In order to get your cohere developer liscense, navigate the the following url [Cohere Dev Access](https://cohere.com/) and click Try Now! in the top right. Once you are signed in, navigate to the API Keys section on the left. Copy the default trial key and paste it into the .env.sample file and change the filename from `.env.sample` to just `.env` so we can use it to authenticate ourselves with Cohere's api.

Optionally, we can install [LangSmith](https://smith.langchain.com/) which will allow us to keep track of our model's calls in a dashboard provided by lanchain. If you want to set this up, navigate to the link and create an account. From there, go to settings and api keys. Create a new personal key and add it to the .env file.

## Lets make our first langchain AI agent!

Setting up the environment variables and imports we will need later

In [13]:
import os
from dotenv import load_dotenv
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
load_dotenv() #Load .env file into the notebook

if os.environ['LANGCHAIN_API_KEY']:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"



In [4]:
model = ChatCohere(model="command-r-plus") #We will use the command r plus model from cohere

When creating prompts to send to the cohere model using langchain, we have access to both a `HumanMessage` and a `SystemMessage`. The human message is similar to what you as a user would type while the system message is more like instructions that the agent will see before every human message. The system message is typically used as a set of instructions about how to answer the human message. See below for an example

In [7]:
messages = [
    SystemMessage(content="Respond with a single word that is most similar to the following word. You must always respond with the word in all caps.  You must only output 1 word for any given request."),
    HumanMessage(content="Banana")
]
model.invoke(messages)

AIMessage(content='PLANT', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c437984f-0057-45f2-9fc1-1480ef4cafbc', 'token_count': {'input_tokens': 241.0, 'output_tokens': 2.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c437984f-0057-45f2-9fc1-1480ef4cafbc', 'token_count': {'input_tokens': 241.0, 'output_tokens': 2.0}}, id='run-327dbacc-45f6-41bb-aacd-91abb8ce6ac8-0', usage_metadata={'input_tokens': 241, 'output_tokens': 2, 'total_tokens': 243})

If you set up LangSmith, you should see the logs of this in the dashboard with the system message, human message, and the visible response.

This output is an AIMessage object, but it contains some meta data that we dont really care about. Lets see how we can get rid it by chaining together some functions.

In [11]:
parser = StrOutputParser()
chain = model | parser


Langchain reserves the `|` operator for chaining together their different objects. Here we chain a model to an output parser, but it can get much more advanced than that. 

In [12]:
chain.invoke(messages)

'PLANT'

Lets try to structure our inputs a bit more with a `ChatPromptTemplate`

In [17]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "italian", "text": "Giraffe"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Giraffe', additional_kwargs={}, response_metadata={})])

In [18]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Giraffe', additional_kwargs={}, response_metadata={})]

Now lets put it on our chain from earlier

In [19]:
chain = prompt_template | model | parser

In [20]:
chain.invoke({"language":"italian", "text":"Hello world"})

'Ciao mondo!'